In [1]:
# Imports
import pandas as pd
from gensim import corpora, models
#from googlesearch import search
import re
import nltk
import chardet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import requests

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/gra248/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gra248/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Clean the abstracts so they dont have HTML tags, special characters, numbers, stopwords, and lemmatize the words for better results in the LDA model
def preprocess_abstract(abstract):
    if abstract is None or not isinstance(abstract, str):
        return []
    abstract = re.sub('<[^<]+?>', '', abstract)  # Remove HTML tags
    abstract = abstract.lower()  # Convert to lowercase
    abstract = re.sub(r'\W+', ' ', abstract)  # Remove special characters and numbers
    words = abstract.split()  # Tokenize
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize words
    return words

In [4]:
# Create a new colunm "Keywords" to be classified into keywords based on the LDA model
def get_lda_keywords(lda_model, bow, num_keywords=8):
    topic_dist = lda_model[bow]
    dominant_topic = max(topic_dist, key=lambda x: x[1])[0]
    topic_terms = lda_model.get_topic_terms(dominant_topic, topn=num_keywords)
    return [dictionary[id] for id, prob in topic_terms]

In [5]:
# This cleans the final abstracts (note the difference is this one is for the final abstracts and the other one is for the abstracts ran for the LDA model they have diffrent processing)
def clean_abstract(abstract):
    if abstract is None or not isinstance(abstract, str):
        return []
    abstract = re.sub('<[^<]+?>', '', abstract)  # Remove HTML tags
    abstract = re.sub(r'\s+', ' ', abstract)  # Remove extra whitespaces
    return abstract.strip()

In [ ]:
# # Load the DOEdata from xls files
# DOE_xls_files = ['DOEdata/DOE_CCF.xls', 'DOEdata/DOE_MRI.xls', 'DOEdata/DOE_OAC.xls']
# for file in DOE_xls_files:
#     df = pd.read_excel(file)
#     # Keep only the relevant columns (abstracts and titles)
#     df = df[['Title', 'Abstract']]
#     # Merge the data frames
#     data_frames.append(df)
print('Data frames loaded', len(data_frames))

In [8]:
# Now we merge them all into one file using DOE NSF or NIH datasets, in the process we will save each segment keyword Ex. NSF_CCF.csv will have a .csv file etc. and a final .csv file of course
NIH_csv_files = [r'/home/gra248/projects/ProjectTrace/NIHdata/NIH_CCF.csv', r'/home/gra248/projects/ProjectTrace/NIHdata/NIH_CICI.csv', r'/home/gra248/projects/ProjectTrace/NIHdata/NIH_CSSI.csv', r'/home/gra248/projects/ProjectTrace/NIHdata/NIH_MRI.csv', r'/home/gra248/projects/ProjectTrace/NIHdata/NIH_OAC.csv']

all_projects = pd.DataFrame()

for file_path in NIH_csv_files:
    data = pd.read_csv(file_path)
    # Keep only the first 8 terms in the "Project Terms" column
    data["Project Terms"] = data["Project Terms"].apply(lambda x: ";".join(x.split(";")[:8]))

    # Extract the first and last name from the "Contact PI / Project Leader" column
    data["First Name"] = data["Contact PI / Project Leader"].apply(lambda x: x.split(",")[1].strip())
    data["Last Name"] = data["Contact PI / Project Leader"].apply(lambda x: x.split(",")[0].strip())

    # Create a new column "PI_contact" by merging the extracted first and last name with the "Organization Name" column
    data["PI_contact"] = data["First Name"] + "." + data["Last Name"] + "@" + data["Organization Name"].str.replace(" ", "").apply(lambda x: x.lower())
    #projects = data[["AwardNumber", "Title", "NSFOrganization", "PrincipalInvestigator", "PIEmailAddress", "Abstract"]]
    projects = data[["Title","Project Terms", "Application ID", "Department", "Contact PI / Project Leader", "PI_contact", "Abstract"]]
    # abstracts = projects["Abstract"].apply(preprocess_abstract)
    # dictionary = corpora.Dictionary(abstracts)
    # corpus = [dictionary.doc2bow(text) for text in abstracts]
    # lda_model = models.LdaModel(corpus, num_topics=8, id2word=dictionary, passes=3)
    # projects["Keyword"] = projects["Abstract"].apply(lambda x: get_lda_keywords(lda_model, dictionary.doc2bow(preprocess_abstract(x))))
    #projects["News"] = projects["Title"].apply(search_news)
    projects["Clean_Abstract"] = projects["Abstract"].apply(clean_abstract)
    
    output = projects.rename(columns={"Title": "Project_name",
                                      "Project Terms": "Keyword",
                                      "Department": "Funding_agency",
                                      "Application ID": "Award_number",
                                      "Contact PI / Project Leader": "PI_name",
                                      "Clean_Abstract": "Description"})
    
    output.drop(columns=["Abstract"], inplace=True)
    output = output[["Project_name", "Funding_agency", "Award_number", "PI_name", "PI_contact", "Keyword", "Description"]]
    output_file = f"{file_path.split('/')[-1].split('.')[0]}_processed.csv"
    output.to_csv(output_file, index=False)
    print(f"Saved processed data to: {output_file}")
    all_projects = pd.concat([all_projects, output], ignore_index=True)

all_projects.to_csv("NIH_all_final.csv", index=False)

/tmp/ipykernel_14466/1460694118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects["Clean_Abstract"] = projects["Abstract"].apply(clean_abstract)
/tmp/ipykernel_14466/1460694118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects["Clean_Abstract"] = projects["Abstract"].apply(clean_abstract)
/tmp/ipykernel_14466/1460694118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

Saved processed data to: NIH_CCF_processed.csv
Saved processed data to: NIH_CICI_processed.csv
Saved processed data to: NIH_CSSI_processed.csv


/tmp/ipykernel_14466/1460694118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects["Clean_Abstract"] = projects["Abstract"].apply(clean_abstract)


Saved processed data to: NIH_MRI_processed.csv
Saved processed data to: NIH_OAC_processed.csv


/tmp/ipykernel_14466/1460694118.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects["Clean_Abstract"] = projects["Abstract"].apply(clean_abstract)
